# Instanzen-Suche
Mit diesem Notebook können alle Instanzen aufgelistet werden, die bestimmte Bedingungen erfüllen. Die Bedingungen können frei gewählt werden. Sie bestehen aus einer Annotationskategorie und einer weiteren Bedingung. Folgende Suchparameter gibt es:
+ nach Wörtern (Lexemen)
+ Nach Wortarten oder Wortartenlisten (benutzt wird das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html) oder [Universal POS tags](https://universaldependencies.org/u/pos/))
+ Nach Anzahl vergebener Kategorietags (z.B.: 3 Moralwerte)
+ Nach bestimmten Annotationslabels (z.B.: Forderer)

Bezüglich der Annotationskategorien stehen Folgende zur Verfügung:
+ *obj_morals*: Alle Moralwerte, die keine subjektiven Ausdrücke sind
+ *subj_morals*: Moralwerte, die subjektive Ausdrücke sind
+ *all_morals*: Alle Moralwerte
+ *protagonists*: Alle Protagonisten-Token
+ *protagonists_doubles*: Alle Protagonisten-Rollen (Achtung: manche Protagonisten-Token werden dabei doppelt gezählt - nämlich, wenn sie mehrere Rollen haben)
+ *com_functions*: Kommunikative Funktionen
+ *expl_demands*: Explizite Forderungen
+ *impl_demands*: Implizite Forderungen
+ *all_demands*: Alle Forderungen

## Setup

Muss ausgeführt werden, wenn das Notebook in Colab ausgeführt wird.
Das Laden der Spacy-Modelle kann länger dauern; man kann es beschleunigen, indem man nur die Sprachen läd, die man benötigt (und die anderen Befehlszeilen löscht).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%git clone https://github.com/maria-becker/Moralization/
%pip install HanTa
%pip install xlsxwriter
%pip install spacy

%python -m spacy download de_core_web_lg
%python -m spacy download en_core_web_lg
%python -m spacy download fr_core_web_lg
%python -m spacy download it_core_web_lg

%cd "/content/Moralization/Annotation Analysis Tools/data_analysis"

Die import-Statements müssen immer ausgeführt werden.

In [1]:
import sys
import label_filtering_collection as lfc
import os

sys.path.append("../_utils_")
import corpus_extraction as ce
import xmi_analysis_util as xau

## Suchfunktionen (Einzelte Korpusdateien)
Code für die Analyse einzelner XMI-Dateien.

Hier bitte eine Liste *file_list* erstellen, die die Dateipfade zu allen Korpusdateien enthält, die ihr analysieren wollt.
+ *Am einfachsten erhält man einen Dateipfad, indem man die Datei im Ordersystem rechtsklickt und dann -> 'Pfad kopieren' auswählt. Und unbedingt den Pfad in Anführungszeichen setzen und davor ein kleines "r"!*
+ *Wer sich ein bisschen auskennt, kann zum Erstellen der Liste auch das os-Modul benutzen, das aus diesem Grund oben importiert wurde.*

Bei der Erstellung des corpus-Objekts ist zu beachten, welche Sprache im Korpus vertreten ist.
    + Deutsch ('de')
    + Englisch ('en')
    + Französisch ('fr')
    + Italienisch ('it')
Mehrsprachliche Korpora werden nicht unterstützt.

In [2]:
filepaths = [
    r"/home/brunobrocai/Desktop/Code/moralization/Testfiles/test_gerichtsurteile_DE.xmi",
]
corpus_collection = ce.Corpus(filepaths, 'ger')

Mit der folgenden Funktion kann nach Lexemen, die als eine bestimmte Kategorie annotiert wurden, gesucht werden.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist das Lexem, das gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der die Wortart gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [4]:
word_labels = lfc.lemma_label_instances(
    corpus_collection,
    "Würde",
    "all_morals",
    hanta=True,
    export=True
)
print(word_labels)

KeyError: (43846, 43875)
{'/home/brunobrocai/Desktop/Code/moralization/Testfiles/test_gerichtsurteile_DE.xmi': []}


Mit der folgenden Funktion kann nach Wortarten, die als eine bestimmte Kategorie annotiert wurden, gesucht werden.
Wird mit HanTa (s.u.) getaggt, nutzt die Funktion das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html). Andernfalls - also mit Spacy - wird [Universal POS tags](https://universaldependencies.org/u/pos/) benutzt, das etwas allgemeiner ist.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist die Wortart, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der das Lexem gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [ ]:
pos_labels = lfc.pos_label_instances(
    corpus_collection,
    "NE",
    "protagonists",
    hanta=False,
    export=False
)
lfc.print_dict(pos_labels)

Mit der folgenden Funktion kann nach einer Liste von Wortarten, die als eine bestimmte Kategorie annotiert wurden, gesucht werden. Dies ist vor allem daher sinnvoll, weil bestimmten Wortarten in den Tagsets mehr als ein Tag zugeordnet ist.
Wird mit HanTa (s.u.) getaggt, nutzt die Funktion das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html). Andernfalls - also mit Spacy - wird [Universal POS tags](https://universaldependencies.org/u/pos/) benutzt, das etwas allgemeiner ist.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist die Wortart, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der die Wortart gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [ ]:
poslist_labels = lfc.poslist_label_instances_collection(
    corpus_collection,
    ["NE", "NN"],
    "protagonists",
    hanta=False,
    export=False
)
lfc.print_dict(poslist_labels)

Mit der folgenden Funktion kann nach Anzahlen von Kategorie-Vergaben gesucht werden. Zum Beispiel können alle Funktionen mit zwei Protagonisten gesucht werden.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter *count* ist die Anzahl, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, die *count* Mal vergeben wurde
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [ ]:
count_labels = lfc.count_label_instances_collection(
    corpus_collection,
    2,
    "protagonists",
    export=False
)
lfc.print_dict(count_labels)

Mit der folgenden Funktion kann nach spezifischen Labeln gesucht werden werden. Zum Beispiel können alle Funktionen herausgesucht werden, in denen ein Protagonist als "Benefizient" markiert wurde.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist das Label, nach dem gesucht werden soll
+ Der dritte Parameter ist die Kategorie, aus der das Label stammt
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [ ]:
tag_labels = lfc.tag_label_instances_collection(
    corpus_collection,
    "Benefizient:in",
    "protagonists",
    export=False
)
lfc.print_dict(tag_labels)